In [ ]:
# --- Spiced-up Gradio UI (Dark, Centered, Smooth) ---
!pip -q install gradio pdfplumber

import os, re, pdfplumber, gradio as gr

# Shared state (kept simple; uses your existing pipeline & retriever)
UI_STATE = {
    "meta": None,        # {"name":..., "size":..., "mime":...}
    "chunks": None,      # list[str]
    "retriever": None,   # Retriever instance
}

# ---------- Helpers ----------
def _human_size(size: int) -> str:
    if size < 1024: return f"{size} B"
    if size < 1024*1024: return f"{size/1024:.1f} KB"
    return f"{size/1024/1024:.2f} MB"

def _ok_file(path: str) -> tuple[bool, str]:
    if not path or not os.path.exists(path): return False, "❌ Could not read uploaded file."
    size = os.path.getsize(path)
    if size > 2 * 1024 * 1024: return False, "❌ File too large (> 2MB)."
    ext = os.path.splitext(path.lower())[1]
    if ext not in (".txt", ".pdf"): return False, "❌ Unsupported file type. Use .txt or .pdf."
    return True, ""

# Robust upload handler (accepts .txt/.pdf, enforces 2MB)
def ui_handle_upload(file):
    if file is None:
        gr.Warning("No file selected.")
        return "❌ No file selected.", [], gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), ""
    path = getattr(file, "name", None)
    ok, msg = _ok_file(path)
    if not ok:
        gr.Warning(msg.replace("❌ ", ""))
        return msg, [], gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), ""
    try:
        text = extract_text(path)  # your function
    except Exception as e:
        gr.Error(f"Failed to read file: {e}")
        return f"❌ Failed to read file: {e}", [], gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), ""
    chunks = chunk_text(text)      # your function
    if not chunks:
        gr.Warning("No readable text found in file.")
        return "❌ No readable text found in file.", [], gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), ""

    size = os.path.getsize(path)
    ext = os.path.splitext(path.lower())[1]
    retr = Retriever(chunks)       # your class

    UI_STATE["meta"] = {"name": os.path.basename(path), "size": size, "mime": ("text/plain" if ext==".txt" else "application/pdf")}
    UI_STATE["chunks"] = chunks
    UI_STATE["retriever"] = retr

    info = f"✅ Loaded: **{UI_STATE['meta']['name']}** • {_human_size(size)} • {len(chunks)} chunk(s)"
    seed_msgs = [{"role":"assistant", "content":"File uploaded. Ask me anything about it."}]
    gr.Info("File uploaded. Ready for questions.")
    # Show chatbot, question box, and ask button; also return a nice file meta line
    meta_line = f"{UI_STATE['meta']['name']} · {_human_size(size)} · {'TXT' if ext=='.txt' else 'PDF'}"
    return info, seed_msgs, gr.update(visible=True), gr.update(visible=True), gr.update(visible=True), meta_line

# Chat handler that calls your ask_question()
def ui_chat_fn(message, history):
    retr = UI_STATE.get("retriever")
    if retr is None:
        return "Please upload a PDF/TXT (≤ 2MB) first."
    q = (message or "").strip()
    if not q:
        return "Ask a question about the uploaded document."
    # Typing indicator: add a transient assistant message (Gradio handles it in UI; final will replace it)
    return ask_question(q, retr)  # your function

# Insert an example into the textbox
def fill_example(text):
    return gr.update(value=text)

# Submit example directly (ask & clear)
def ask_example(text, history):
    if not text.strip():
        return "", history
    ans = ui_chat_fn(text, history)
    new_hist = history + [{"role":"user","content":text}, {"role":"assistant","content":ans}]
    return "", new_hist

# ---------- UI ----------
custom_css = """
.gradio-container { max-width: 860px !important; margin: 0 auto !important; }
footer { display: none !important; }
#app-title { text-align: center; }
#meta-line { color: #94a3b8; font-size: 13px; text-align: center; margin-top: -10px; }
#drop-area { border: 1.5px dashed #1f2937; border-radius: 14px; background: #0b1220; }
#drop-area:hover { border-color: #2a3a60; }
#ask-row button { transition: transform .08s ease, filter .2s ease; }
#ask-row button:active { transform: translateY(1px); }
#examples .gr-button { background: #0b1220; border-color: #1f2937; }
#examples .gr-button:hover { filter: brightness(1.08); }
"""

with gr.Blocks(theme=gr.themes.Soft(), css=custom_css) as demo:
    gr.Markdown("## 📄 Document Q&A — Dark & Centered", elem_id="app-title")
    meta_line = gr.Markdown("", elem_id="meta-line")

    with gr.Row():
        with gr.Column(scale=1):
            drop = gr.File(label="Upload PDF/TXT (≤ 2MB)", file_count="single", elem_id="drop-area")
        with gr.Column(scale=1):
            info = gr.Markdown("No file loaded yet.")

    chatbot = gr.Chatbot(height=360, type="messages", show_copy_button=True, avatar_images=(None, None), bubble_full_width=False, layout="bubble")
    question = gr.Textbox(placeholder="Type your question…", autofocus=False)
    with gr.Row(elem_id="ask-row"):
        ask_btn = gr.Button("Ask", variant="primary", size="lg")
        clear_btn = gr.Button("Clear Chat", variant="secondary")

    # Quick example prompts (nice for demoing)
    gr.Markdown("#### Examples")
    with gr.Row(elem_id="examples"):
        ex1 = gr.Button("Summarize the document")
        ex2 = gr.Button("List the key points")
        ex3 = gr.Button("What is the main argument?")
        ex4 = gr.Button("Quote the definition of X")

    # Hide chat inputs until a file is uploaded
    chatbot.visible = False
    question.visible = False
    ask_btn.visible = False

    # Wire upload
    drop.upload(
        fn=ui_handle_upload,
        inputs=drop,
        outputs=[info, chatbot, chatbot, question, ask_btn, meta_line]
    )

    # Wire Q&A
    def submit_and_clear(q, history):
        # Show a temporary "assistant is typing…" bubble
        typing = {"role":"assistant", "content":"…thinking"}
        history = history + [{"role":"user","content":q}, typing]
        ans = ui_chat_fn(q, history[:-1])  # don't include typing in the model call
        history = history[:-1] + [{"role":"assistant","content":ans}]
        return "", history

    question.submit(submit_and_clear, inputs=[question, chatbot], outputs=[question, chatbot])
    ask_btn.click(submit_and_clear, inputs=[question, chatbot], outputs=[question, chatbot])

    # Clear chat
    def clear_chat():
        return []
    clear_btn.click(clear_chat, inputs=None, outputs=chatbot)

    # Example buttons: either fill textbox or ask directly
    ex1.click(ask_example, inputs=ex1, outputs=[question, chatbot])
    ex2.click(ask_example, inputs=ex2, outputs=[question, chatbot])
    ex3.click(ask_example, inputs=ex3, outputs=[question, chatbot])
    ex4.click(fill_example, inputs=ex4, outputs=question)

demo.launch()
